In [12]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [13]:
!python3 -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 4.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [14]:
import parser as p
import util as u

recipe = p.get_recipe_from_url("https://www.allrecipes.com/recipe/19644/moussaka/")
# recipe = p.get_recipe_from_url("https://www.allrecipes.com/recipe/218091/classic-and-simple-meat-lasagna/")
# recipe = p.get_recipe_from_url("https://www.allrecipes.com/recipe/230238/gingerbread-men-cookies/")
print(recipe.title, '\n')
for ingr in recipe.ingredients:
    print(u.fraction_to_str(ingr.quantity) if ingr.quantity else '',
          ' ' if ingr.quantity else '', ingr.unit if ingr.unit else '',
          ' ' if ingr.unit else '', ingr.name, sep='')
print()
for step in recipe.steps:
    print(step.text, step.methods, [(ingr.name, ingr.quantity) for ingr in step.ingredients],
          step.tools, step.times, step.temps)

Moussaka 

3 eggplants, peeled and cut lengthwise into 1/2 inch thick slices
salt to taste
1/4 cup olive oil
1 tablespoon butter
1 pound lean ground beef
2 onions, chopped
1 clove garlic, minced
ground black pepper to taste
2 tablespoons dried parsley
1/2 teaspoon fines herbs
1/4 teaspoon ground cinnamon
1/2 teaspoon ground nutmeg, divided
1 (8 ounce) can tomato sauce
1/2 cup red wine
1 egg, beaten
4 cups milk
1/2 cup butter
6 tablespoons all-purpose flour
ground white pepper, to taste
1 1/2 cups freshly grated Parmesan cheese

Lay eggplant slices on paper towels. ['lay'] [('eggplant slices', Fraction(3, 1))] [] [] []
Sprinkle lightly with salt. ['sprinkle'] [('salt', None)] [] [] []
Let sit for 30 minutes to draw out moisture. ['sit'] [] [] ['30 minutes'] []
Pat dry with paper towels. ['dry'] [] [] [] []
Warm olive oil in a skillet over high heat. ['warm'] [('warm olive oil', Fraction(1, 4))] ['a skillet'] [] ['high heat']
Fry eggplant until browned, 2 to 3 minutes per side. ['fry'] [

In [15]:
# text = "lay eggplant slices on paper towels"
text = "eggplant slices"
# text = "cook and stir until beef is browned, 8 to 10 minutes"
# text = "scald milk in a saucepan over medium heat"
# text = "meanwhile, place ground beef, garlic, oregano, garlic powder, salt, and black pepper in a large skillet over medium heat"
# text = "add lasagna noodles and cook for 10 minutes or until al dente"
# text = "season béchamel sauce with salt and white pepper"
# text = "serve hot and enjoy!"
# text = "bake in the preheated oven until edges of cookies are set and just begin to brown, 8 to 10 minutes"
# text = "pour béchamel sauce on top and sprinkle with remaining 1/4 teaspoon nutmeg"
# text = "cover with remaining eggplant and sprinkle another 1/2 cup cheese on top"
# text = "add parsley, fines herbs, cinnamon, and 1/4 teaspoon nutmeg"

doc = u.nlp(text)
for token in doc:
    print(token.text, token.pos_, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children], token.ent_type_)
print()
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text, chunk.start, chunk.end, chunk.root.i)
print()
for ent in doc.ents:
    print(ent.text, ent.label_)

eggplant NOUN compound slices NOUN [] 
slices NOUN ROOT slices NOUN [eggplant] 

eggplant slices slices ROOT slices 0 2 1



In [16]:
from nltk.corpus import wordnet as wn

def print_hypernym_paths(noun: str):
    mix_sets = wn.synsets(noun, wn.NOUN)
    print(mix_sets)
    for s in mix_sets:
        print(s.hypernym_paths())
    print()

def print_verb_paths(verb: str):
    sets = wn.synsets(verb, wn.VERB)
    print(sets)
    for s in sets:
        print(s.hypernym_paths())
    print()

print_verb_paths("scald")
print_verb_paths("bake")
print_verb_paths("saute")
print(u.NounType.from_str("season"))
print(u.NounType.from_str("tin"))
print(u.NounType.from_str("mixer"))
print(u.NounType.from_str("men"))

[Synset('blister.v.02'), Synset('scald.v.02'), Synset('scald.v.03'), Synset('scald.v.04')]
[[Synset('express.v.02'), Synset('state.v.01'), Synset('note.v.01'), Synset('comment.v.01'), Synset('knock.v.06'), Synset('attack.v.02'), Synset('blister.v.02')]]
[[Synset('change.v.01'), Synset('affect.v.01'), Synset('process.v.01'), Synset('scald.v.02')]]
[[Synset('change.v.01'), Synset('heat.v.01'), Synset('scald.v.03')]]
[[Synset('change.v.01'), Synset('damage.v.01'), Synset('burn.v.15'), Synset('scald.v.04')]]

[Synset('bake.v.01'), Synset('bake.v.02'), Synset('broil.v.02'), Synset('bake.v.04')]
[[Synset('change.v.02'), Synset('change_integrity.v.01'), Synset('cook.v.03'), Synset('bake.v.01')]]
[[Synset('make.v.03'), Synset('create_from_raw_material.v.01'), Synset('bake.v.02')]]
[[Synset('change.v.01'), Synset('heat.v.01'), Synset('broil.v.02')]]
[[Synset('be.v.01'), Synset('bake.v.04')]]

[Synset('saute.v.01')]
[[Synset('change.v.02'), Synset('change_integrity.v.01'), Synset('cook.v.03'), S